# Demo for using C3S intake catalog

Intake Example:
https://github.com/intake/intake-examples/blob/master/tutorial/data_scientist.ipynb


In [1]:
import intake

## Open remote catalog

In [2]:
cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"

cat = intake.open_catalog(cat_url)
list(cat)

['c3s-cmip5',
 'c3s-cmip5-daily-pressure-level',
 'c3s-cmip5-daily-single-level',
 'c3s-cmip5-monthly-pressure-level',
 'c3s-cmip5-monthly-single-level',
 'c3s-cmip6',
 'c3s-cmip6-r3',
 'c3s-cmip6-decadal',
 'c3s-cmip6-decadal-dwd',
 'c3s-cordex',
 'c3s-ipcc-atlas']

## Load catalog for c3s-cmip6
Catalogs will be cached locally in `~/.intake/cache`.

See: https://intake.readthedocs.io/en/latest/catalog.html?highlight=simplecache#caching-source-files-locally

In [3]:
df_cmip6 = cat['c3s-cmip6'].read()
df_cmip6

,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level
0,c3s-cmip6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp245....,ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/r1i1p1f2/A...,28037112,c3s-cmip6,ScenarioMIP,MOHC,UKESM1-0-LL,ssp245,r1i1p1f2,Amon,ts,gn,v20190507,2015-01-16T00:00:00,2049-12-16T00:00:00,"0.94, -89.38, 359.06, 89.38",NaN
1,c3s-cmip6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp245....,ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/r1i1p1f2/A...,38838222,c3s-cmip6,ScenarioMIP,MOHC,UKESM1-0-LL,ssp245,r1i1p1f2,Amon,ts,gn,v20190507,2050-01-16T00:00:00,2100-12-16T00:00:00,"0.94, -89.38, 359.06, 89.38",NaN
2,c3s-cmip6.ScenarioMIP.NCAR.CESM2.ssp370.r4i1p1...,ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/pr...,104081588,c3s-cmip6,ScenarioMIP,NCAR,CESM2,ssp370,r4i1p1f1,Amon,pr,gn,v20200528,2015-01-15T12:00:00,2064-12-15T12:00:00,"0.00, -90.00, 358.75, 90.00",NaN
3,c3s-cmip6.ScenarioMIP.NCAR.CESM2.ssp370.r4i1p1...,ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/pr...,74977662,c3s-cmip6,ScenarioMIP,NCAR,CESM2,ssp370,r4i1p1f1,Amon,pr,gn,v20200528,2065-01-15T12:00:00,2100-12-15T12:00:00,"0.00, -90.00, 358.75, 90.00",NaN
4,c3s-cmip6.ScenarioMIP.AS-RCEC.TaiESM1.ssp370.r...,ScenarioMIP/AS-RCEC/TaiESM1/ssp370/r1i1p1f1/Am...,144277888,c3s-cmip6,ScenarioMIP,AS-RCEC,TaiESM1,ssp370,r1i1p1f1,Amon,rlut,gn,v20201014,2015-01-16T12:00:00,2100-12-16T12:00:00,"0.00, -90.00, 358.75, 90.00",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133183,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,204026121,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,tauu,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",NaN
133184,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,164344947,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,rlus,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",NaN
133185,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,3420481903,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,ua,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",100.00 500.00 1000.00 2000.00 3000.00 5000.00 ...
133186,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,143468073,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,tasmin,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",2.00


### Search dataset ...

In [4]:
df = df_cmip6.loc[
    (df_cmip6.experiment_id=="historical")
    & (df_cmip6.member_id=="r1i1p4f2")
    & (df_cmip6.source_id=="MIROC-ES2H")
]
df.head()

,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level
133156,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,143258359,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,tas,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",2.00
133157,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,207232287,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,vas,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",10.00
133158,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,144450061,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,ts,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",NaN
133159,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,3085501617,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,hur,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",100.00 500.00 1000.00 2000.00 3000.00 5000.00 ...
133160,c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i...,CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/Amon...,3717607962,c3s-cmip6,CMIP,MIROC,MIROC-ES2H,historical,r1i1p4f2,Amon,va,gn,v20220322,1850-01-16T12:00:00,2014-12-16T12:00:00,"0.00, -88.93, 358.59, 88.93",100.00 500.00 1000.00 2000.00 3000.00 5000.00 ...


In [5]:
df.ds_id.unique()

array(['c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.tas.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.vas.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.ts.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.hur.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.va.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.rsds.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.uas.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.ps.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.hfls.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.hfss.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.rlut.gn.v20220322',
       'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.rsut.gn.v2

## Load catalog for c3s-cmip6-decadal

In [6]:
df_cmip6_decadal = cat['c3s-cmip6-decadal'].read()
df_cmip6_decadal

,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level,member_year,member_ensemble,prefix
0,c3s-cmip6-decadal.DCPP.MPI-M.MPI-ESM1-2-HR.dcp...,DCPP/MPI-M/MPI-ESM1-2-HR/dcppA-hindcast/s2016-...,492479692,c3s-cmip6-decadal,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,s2016-r8i1p1f1,day,tasmax,gn,v20210111,2016-11-01T12:00:00,2026-12-31T12:00:00,"0.00, -89.28, 359.06, 89.28",2.00,s2016,r8i1p1f1,c3s-cmip6-decadal
1,c3s-cmip6-decadal.DCPP.MPI-M.MPI-ESM1-2-HR.dcp...,DCPP/MPI-M/MPI-ESM1-2-HR/dcppA-hindcast/s2009-...,737204109,c3s-cmip6-decadal,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,s2009-r8i1p1f1,day,pr,gn,v20210107,2009-11-01T12:00:00,2019-12-31T12:00:00,"0.00, -89.28, 359.06, 89.28",NaN,s2009,r8i1p1f1,c3s-cmip6-decadal
2,c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dc...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1987...,953384,c3s-cmip6-decadal,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s1987-r3i1p1f2,Amon,pr,gn,v20200417,1987-11-16T00:00:00,1987-12-16T00:00:00,"0.42, -89.72, 359.58, 89.72",NaN,s1987,r3i1p1f2,c3s-cmip6-decadal
3,c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dc...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1995...,603757,c3s-cmip6-decadal,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s1995-r6i1p1f2,Amon,tas,gn,v20200417,1995-11-16T00:00:00,1995-12-16T00:00:00,"0.42, -89.72, 359.58, 89.72",1.50,s1995,r6i1p1f2,c3s-cmip6-decadal
4,c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dc...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s2004...,14824913,c3s-cmip6-decadal,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s2004-r10i1p1f2,AERday,zg500,gn,v20200417,2004-11-01T12:00:00,2004-12-30T12:00:00,"0.42, -89.72, 359.58, 89.72",50000.00,s2004,r10i1p1f2,c3s-cmip6-decadal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102225,c3s-cmip6-decadal.DCPP.DWD.MPI-ESM1-2-LR.dcppA...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s1995-r1...,4119531,c3s-cmip6-decadal,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s1995-r16i1p1f1,Amon,tas,gn,v20220126,1995-11-30T18:00:00,2005-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",2.00,s1995,r16i1p1f1,c3s-cmip6-decadal
102226,c3s-cmip6-decadal.DCPP.DWD.MPI-ESM1-2-LR.dcppA...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s1973-r1...,6635233,c3s-cmip6-decadal,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s1973-r16i1p1f1,Amon,pr,gn,v20220126,1973-11-30T18:00:00,1983-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",NaN,s1973,r16i1p1f1,c3s-cmip6-decadal
102227,c3s-cmip6-decadal.DCPP.DWD.MPI-ESM1-2-LR.dcppA...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s2014-r8...,4114536,c3s-cmip6-decadal,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s2014-r8i1p1f1,Amon,tas,gn,v20220126,2014-11-30T18:00:00,2024-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",2.00,s2014,r8i1p1f1,c3s-cmip6-decadal
102228,c3s-cmip6-decadal.DCPP.DWD.MPI-ESM1-2-LR.dcppA...,DCPP/DWD/MPI-ESM1-2-LR/dcppA-hindcast/s2006-r1...,6641928,c3s-cmip6-decadal,DCPP,DWD,MPI-ESM1-2-LR,dcppA-hindcast,s2006-r14i1p1f1,Amon,pr,gn,v20220126,2006-11-30T18:00:00,2016-12-31T18:00:00,"0.00, -88.57, 358.12, 88.57",NaN,s2006,r14i1p1f1,c3s-cmip6-decadal


In [7]:
df = df_cmip6_decadal.loc[
    (df_cmip6_decadal.variable_id=="tas") 
    & (df_cmip6_decadal.experiment_id=="dcppA-hindcast")
    & (df_cmip6_decadal.table_id=="Amon")
    & (df_cmip6_decadal.institution_id=="MOHC")
    & (df_cmip6_decadal.member_year=="s1995")
]
df.nunique()

ds_id               10
path               120
size               120
mip_era              1
activity_id          1
institution_id       1
source_id            1
experiment_id        1
member_id           10
table_id             1
variable_id          1
grid_label           1
version              1
start_time          12
end_time            12
bbox                 1
level                1
member_year          1
member_ensemble     10
prefix               1
dtype: int64

In [8]:
ds_ids = list(df.ds_id)
ds_ids.sort()
ds_ids

['c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v20200417',
 'c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dcppA-hindcast.s1995-r10i1p1f2.Amon.tas.gn.v2

## Load Catalog for C3S-CORDEX

In [9]:
df_cordex = cat['c3s-cordex'].read()
df_cordex

,ds_id,path,size,project,product,domain,institute,driving_model,experiment_id,ensemble,rcm_name,rcm_version,time_frequency,variable,version,start_time,end_time,bbox,level
0,c3s-cordex.output.EAS-44.CLMcom.CNRM-CERFACS-C...,output/EAS-44/CLMcom/CNRM-CERFACS-CNRM-CM5/rcp...,176452637,c3s-cordex,output,EAS-44,CLMcom,CNRM-CERFACS-CNRM-CM5,rcp45,r1i1p1,CLMcom-CCLM5-0-2,v1,day,huss,v20201215,2006-01-01T12:00:00,2010-12-31T12:00:00,"63.36, -18.23, 175.13, 58.59",10.00
1,c3s-cordex.output.EAS-44.CLMcom.CNRM-CERFACS-C...,output/EAS-44/CLMcom/CNRM-CERFACS-CNRM-CM5/rcp...,176393262,c3s-cordex,output,EAS-44,CLMcom,CNRM-CERFACS-CNRM-CM5,rcp45,r1i1p1,CLMcom-CCLM5-0-2,v1,day,huss,v20201215,2011-01-01T12:00:00,2015-12-31T12:00:00,"63.36, -18.23, 175.13, 58.59",10.00
2,c3s-cordex.output.EAS-44.CLMcom.CNRM-CERFACS-C...,output/EAS-44/CLMcom/CNRM-CERFACS-CNRM-CM5/rcp...,176557045,c3s-cordex,output,EAS-44,CLMcom,CNRM-CERFACS-CNRM-CM5,rcp45,r1i1p1,CLMcom-CCLM5-0-2,v1,day,huss,v20201215,2016-01-01T12:00:00,2020-12-31T12:00:00,"63.36, -18.23, 175.13, 58.59",10.00
3,c3s-cordex.output.EAS-44.CLMcom.CNRM-CERFACS-C...,output/EAS-44/CLMcom/CNRM-CERFACS-CNRM-CM5/rcp...,176346476,c3s-cordex,output,EAS-44,CLMcom,CNRM-CERFACS-CNRM-CM5,rcp45,r1i1p1,CLMcom-CCLM5-0-2,v1,day,huss,v20201215,2021-01-01T12:00:00,2025-12-31T12:00:00,"63.36, -18.23, 175.13, 58.59",10.00
4,c3s-cordex.output.EAS-44.CLMcom.CNRM-CERFACS-C...,output/EAS-44/CLMcom/CNRM-CERFACS-CNRM-CM5/rcp...,176260737,c3s-cordex,output,EAS-44,CLMcom,CNRM-CERFACS-CNRM-CM5,rcp45,r1i1p1,CLMcom-CCLM5-0-2,v1,day,huss,v20201215,2026-01-01T12:00:00,2030-12-31T12:00:00,"63.36, -18.23, 175.13, 58.59",10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567374,c3s-cordex.output.CAM-44.SMHI.ICHEC-EC-EARTH.h...,output/CAM-44/SMHI/ICHEC-EC-EARTH/historical/r...,60776141,c3s-cordex,output,CAM-44,SMHI,ICHEC-EC-EARTH,historical,r12i1p1,SMHI-RCA4,v1,day,tasmin,v20201201,1981-01-01T12:00:00,1985-12-31T12:00:00,"-127.58, -19.46, -19.41, 46.38",2.00
567375,c3s-cordex.output.CAM-44.SMHI.ICHEC-EC-EARTH.h...,output/CAM-44/SMHI/ICHEC-EC-EARTH/historical/r...,60562273,c3s-cordex,output,CAM-44,SMHI,ICHEC-EC-EARTH,historical,r12i1p1,SMHI-RCA4,v1,day,tasmin,v20201201,1986-01-01T12:00:00,1990-12-31T12:00:00,"-127.58, -19.46, -19.41, 46.38",2.00
567376,c3s-cordex.output.CAM-44.SMHI.ICHEC-EC-EARTH.h...,output/CAM-44/SMHI/ICHEC-EC-EARTH/historical/r...,60779301,c3s-cordex,output,CAM-44,SMHI,ICHEC-EC-EARTH,historical,r12i1p1,SMHI-RCA4,v1,day,tasmin,v20201201,1991-01-01T12:00:00,1995-12-31T12:00:00,"-127.58, -19.46, -19.41, 46.38",2.00
567377,c3s-cordex.output.CAM-44.SMHI.ICHEC-EC-EARTH.h...,output/CAM-44/SMHI/ICHEC-EC-EARTH/historical/r...,60957749,c3s-cordex,output,CAM-44,SMHI,ICHEC-EC-EARTH,historical,r12i1p1,SMHI-RCA4,v1,day,tasmin,v20201201,1996-01-01T12:00:00,2000-12-31T12:00:00,"-127.58, -19.46, -19.41, 46.38",2.00


In [10]:
df = df_cordex.loc[
    (df_cordex.variable=="tas") 
    & (df_cordex.experiment_id=="rcp85")
    & (df_cordex.domain=="EUR-11")
    & (df_cordex.time_frequency=="mon")
    & (df_cordex.driving_model=="MOHC-HadGEM2-ES")
    & (df_cordex.ensemble=="r1i1p1")
]
df.head()

,ds_id,path,size,project,product,domain,institute,driving_model,experiment_id,ensemble,rcm_name,rcm_version,time_frequency,variable,version,start_time,end_time,bbox,level
11817,c3s-cordex.output.EUR-11.MOHC.MOHC-HadGEM2-ES....,output/EUR-11/MOHC/MOHC-HadGEM2-ES/rcp85/r1i1p...,27812779,c3s-cordex,output,EUR-11,MOHC,MOHC-HadGEM2-ES,rcp85,r1i1p1,MOHC-HadREM3-GA7-05,v1,mon,tas,v20200330,2005-12-16T00:00:00,2010-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.59",1.50
11818,c3s-cordex.output.EUR-11.MOHC.MOHC-HadGEM2-ES....,output/EUR-11/MOHC/MOHC-HadGEM2-ES/rcp85/r1i1p...,46090612,c3s-cordex,output,EUR-11,MOHC,MOHC-HadGEM2-ES,rcp85,r1i1p1,MOHC-HadREM3-GA7-05,v1,mon,tas,v20200330,2011-01-16T00:00:00,2020-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.59",1.50
11819,c3s-cordex.output.EUR-11.MOHC.MOHC-HadGEM2-ES....,output/EUR-11/MOHC/MOHC-HadGEM2-ES/rcp85/r1i1p...,46067468,c3s-cordex,output,EUR-11,MOHC,MOHC-HadGEM2-ES,rcp85,r1i1p1,MOHC-HadREM3-GA7-05,v1,mon,tas,v20200330,2021-01-16T00:00:00,2030-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.59",1.50
11820,c3s-cordex.output.EUR-11.MOHC.MOHC-HadGEM2-ES....,output/EUR-11/MOHC/MOHC-HadGEM2-ES/rcp85/r1i1p...,46059570,c3s-cordex,output,EUR-11,MOHC,MOHC-HadGEM2-ES,rcp85,r1i1p1,MOHC-HadREM3-GA7-05,v1,mon,tas,v20200330,2031-01-16T00:00:00,2040-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.59",1.50
11821,c3s-cordex.output.EUR-11.MOHC.MOHC-HadGEM2-ES....,output/EUR-11/MOHC/MOHC-HadGEM2-ES/rcp85/r1i1p...,46034891,c3s-cordex,output,EUR-11,MOHC,MOHC-HadGEM2-ES,rcp85,r1i1p1,MOHC-HadREM3-GA7-05,v1,mon,tas,v20200330,2041-01-16T00:00:00,2050-12-16T00:00:00,"-44.59, 21.99, 64.96, 72.59",1.50


In [11]:
collection = df.ds_id.unique()[0]
collection

'c3s-cordex.output.EUR-11.MOHC.MOHC-HadGEM2-ES.rcp85.r1i1p1.MOHC-HadREM3-GA7-05.v1.mon.tas.v20200330'

## Load Catalog for C3S-CMIP5

In [12]:
df_cmip5 = cat['c3s-cmip5'].read()
df_cmip5

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
0,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,1,Total cloud fraction (clt),"Total refers to the whole atmosphere column, a...",https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
1,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m^-2.s^-1,Evaporation (evspsbl),Evaporation rate. It includes conversion to va...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
2,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface latent heat flux (hfls),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
3,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,W.m^-2,Surface sensible heat flux (hfss),Flux per unit area of heat between the surface...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
4,c3s-cmip5.output1.BCC.bcc-csm1-1.amip.mon.atmo...,output1/BCC/bcc-csm1-1/amip/mon/atmos/Amon/r1i...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1,amip,mon,atmos,...,v20181201,1979-01-01T12:00:00,2008-12-31T12:00:00,NaN,NaN,kg.m-^2.s^-1,Snowfall flux (prsn),Mass of water in the form of snow precipitatin...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171682,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2004-01-01T12:00:00,2004-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171683,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2005-01-01T12:00:00,2005-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171684,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2006-01-01T12:00:00,2006-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level
171685,c3s-cmip5.output1.NOAA-GFDL.GFDL-HIRAM-C360.am...,output1/NOAA-GFDL/GFDL-HIRAM-C360/amip/day/atm...,NaN,c3s-cmip5,output1,NOAA-GFDL,GFDL-HIRAM-C360,amip,day,atmos,...,v20110601,2007-01-01T12:00:00,2007-12-31T12:00:00,NaN,NaN,m,Geopotential height (zg),Gravitational potential energy per unit mass n...,https://data.mips.copernicus-climate.eu/thredd...,cmip5-daily-pressure-level


In [13]:
df = df_cmip5.loc[
    (df_cmip5.variable=="tas") 
    & (df_cmip5.experiment=="rcp85")
    & (df_cmip5.time_frequency=="mon")
    & (df_cmip5.model=="bcc-csm1-1-m")
    & (df_cmip5.ensemble=="r1i1p1")
]
df.head()

,ds_id,path,size,project,product,institute,model,experiment,time_frequency,realm,...,version,start_time,end_time,bbox,level,units,variable_name,description,url,label
546,c3s-cmip5.output1.BCC.bcc-csm1-1-m.rcp85.mon.a...,output1/BCC/bcc-csm1-1-m/rcp85/mon/atmos/Amon/...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,rcp85,mon,atmos,...,v20181201,2006-01-01T12:00:00,2099-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level
547,c3s-cmip5.output1.BCC.bcc-csm1-1-m.rcp85.mon.a...,output1/BCC/bcc-csm1-1-m/rcp85/mon/atmos/Amon/...,NaN,c3s-cmip5,output1,BCC,bcc-csm1-1-m,rcp85,mon,atmos,...,v20181201,2100-01-01T12:00:00,2100-12-31T12:00:00,NaN,NaN,K,Near-surface (2m) air temperature (tas),Temperature of the air near the surface.,https://data.mips.copernicus-climate.eu/thredd...,cmip5-monthly-single-level


In [14]:
collection = df.ds_id.unique()[0]
collection

'c3s-cmip5.output1.BCC.bcc-csm1-1-m.rcp85.mon.atmos.Amon.r1i1p1.tas.v20181201'

## Load Catalog for C3S-ATLAS

In [15]:
df_atlas = cat['c3s-ipcc-atlas'].read()
df_atlas

,ds_id,path,size,project,domain,experiment,time_frequency,variable,start_time,end_time,bbox,level,prefix
0,c3s-ipcc-atlas.fd.CMIP5.rcp45.mon,CMIP5/rcp45/fd_CMIP5_rcp45_mon_200601-210012.nc,206830356,CMIP5,NaN,rcp45,mon,fd,2006-01-01T00:00:00,2100-12-01T00:00:00,"-179.00, -89.00, 179.00, 89.00",2.00,c3s-ipcc-atlas
1,c3s-ipcc-atlas.tnn.CMIP5.rcp45.mon,CMIP5/rcp45/tnn_CMIP5_rcp45_mon_200601-210012.nc,899056159,CMIP5,NaN,rcp45,mon,tnn,2006-01-01T00:00:00,2100-12-01T00:00:00,"-179.00, -89.00, 179.00, 89.00",2.00,c3s-ipcc-atlas
2,c3s-ipcc-atlas.tx40.CMIP5.rcp45.mon,CMIP5/rcp45/tx40_CMIP5_rcp45_mon_200601-210012.nc,52647634,CMIP5,NaN,rcp45,mon,tx40,2006-01-01T00:00:00,2100-12-01T00:00:00,"-179.00, -89.00, 179.00, 89.00",2.00,c3s-ipcc-atlas
3,c3s-ipcc-atlas.tx35ba.CMIP5.rcp45.mon,CMIP5/rcp45/tx35ba_CMIP5_rcp45_mon_200601-2100...,40939549,CMIP5,NaN,rcp45,mon,tx35ba,2006-01-01T00:00:00,2100-12-01T00:00:00,"-179.00, -89.00, 179.00, 89.00",2.00,c3s-ipcc-atlas
4,c3s-ipcc-atlas.rx1day.CMIP5.rcp45.mon,CMIP5/rcp45/rx1day_CMIP5_rcp45_mon_200601-2100...,1002850075,CMIP5,NaN,rcp45,mon,rx1day,2006-01-01T00:00:00,2100-12-01T00:00:00,"-179.00, -89.00, 179.00, 89.00",NaN,c3s-ipcc-atlas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,c3s-ipcc-atlas.tx35.CORDEX-WAS.rcp85.mon,CORDEX-WAS/rcp85/tx35_CORDEX-WAS_rcp85_mon_200...,315822711,CORDEX,WAS,rcp85,mon,tx35,2006-01-01T00:00:00,2100-12-01T00:00:00,"19.25, -15.75, 116.25, 45.75",2.00,c3s-ipcc-atlas
859,c3s-ipcc-atlas.spi6.CORDEX-WAS.rcp85.mon,CORDEX-WAS/rcp85/spi6_CORDEX-WAS_rcp85_mon_200...,877566524,CORDEX,WAS,rcp85,mon,spi6,2006-01-01T00:00:00,2100-12-01T00:00:00,"19.25, -15.75, 116.25, 45.75",NaN,c3s-ipcc-atlas
860,c3s-ipcc-atlas.rx5day.CORDEX-WAS.rcp85.mon,CORDEX-WAS/rcp85/rx5day_CORDEX-WAS_rcp85_mon_2...,1107127612,CORDEX,WAS,rcp85,mon,rx5day,2006-01-01T00:00:00,2100-12-01T00:00:00,"19.25, -15.75, 116.25, 45.75",NaN,c3s-ipcc-atlas
861,c3s-ipcc-atlas.tn.CORDEX-WAS.rcp85.mon,CORDEX-WAS/rcp85/tn_CORDEX-WAS_rcp85_mon_20060...,853394968,CORDEX,WAS,rcp85,mon,tn,2006-01-01T00:00:00,2100-12-01T00:00:00,"19.25, -15.75, 116.25, 45.75",2.00,c3s-ipcc-atlas


In [16]:
df_atlas.nunique()

ds_id             863
path              863
size              863
project             3
domain             11
experiment          8
time_frequency      2
variable           22
start_time          5
end_time            7
bbox               14
level               2
prefix              1
dtype: int64